In [153]:
#import package and file
import pandas as pd
import numpy as np

student_data = pd.read_csv("Resource/students_complete.csv")
school_data = pd.read_csv("Resource/schools_complete.csv")

In [154]:
# Calculate the total number of schools
TotalNumber_School = school_data['School ID'].nunique()

# Calculate the total number of students
TotalNumber_Student = student_data['Student ID'].nunique()

# Calculate the total budget
Total_budget = school_data['budget'].sum()

# Calculate the average math score 
avg_MathScore = student_data['math_score'].mean()

# Calculate the average reading score
avg_ReadingScore = student_data['reading_score'].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (avg_MathScore+avg_ReadingScore)/2

# Calculate the percentage of students with a passing math score (70 or greater)
student_passing_math = student_data[student_data.math_score >= 70].count()
student_passing_math_count = student_passing_math['Student ID']
per_pass_math = student_passing_math_count/TotalNumber_Student *100

# Calculate the percentage of students with a passing reading score (70 or greater)
student_passing_reading = student_data[student_data.reading_score >= 70].count()
student_passing_reading_count = student_passing_reading['Student ID']
per_pass_reading = student_passing_reading_count/TotalNumber_Student *100

# Create a dataframe to hold the above results
District_Summary = pd.DataFrame ({'Total Schools':[TotalNumber_School],'Total Student':[TotalNumber_Student],'Total Budget':[Total_budget],\
                                 'Average Math Score':[avg_MathScore],'Average Reading Score':[avg_ReadingScore],\
                                 '% Passing Math':[per_pass_math],'% Passing Reading':[per_pass_reading],\
                                 '% Overall Passing Rate':[overall_passing_rate]})

# Optional: give the displayed data cleaner formatting
District_Summary_format = pd.DataFrame(District_Summary)
District_Summary_format['Total Student']=District_Summary_format['Total Student'].map('{:,}'.format)
District_Summary_format['Total Budget']=District_Summary_format['Total Budget'].map('${:,.2f}'.format)

#print
District_Summary_format

,Total Schools,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [155]:
# School Summary

# create column in student table to separate student whose math >=70 from math <70
student_data.loc[student_data.math_score < 70,'pass_math']=0
student_data.loc[student_data.math_score >= 70,'pass_math']=1

# create column in student table to separate student whose reading >=70 from reading <70
student_data.loc[student_data.reading_score < 70,'pass_reading']=0
student_data.loc[student_data.reading_score >= 70,'pass_reading']=1

# group student by school, and aggregate count, average
stu_by_school = student_data.groupby(['school_name']).agg({'Student ID':'count','math_score':'mean', \
                                                           'reading_score':'mean','pass_math':'sum','pass_reading':'sum' })

# calculation % of pass
stu_by_school['% Passing Math']= stu_by_school['pass_math'] / stu_by_school['Student ID'] *100
stu_by_school['% Passing Reading']= stu_by_school['pass_reading'] / stu_by_school['Student ID'] *100
stu_by_school['% Overall Passing Rate'] = (stu_by_school['% Passing Math'] + stu_by_school['% Passing Reading'])/2
stu_by_school= stu_by_school.reset_index()

#merge with school table to get school info
stu_by_school_merge = school_data.merge(stu_by_school, on='school_name')
stu_by_school_merge['Per Student Budget']=stu_by_school_merge['budget'] / stu_by_school_merge['Student ID']
stu_by_school_merge=stu_by_school_merge.set_index('school_name')

#choose columns to display
school_summary = stu_by_school_merge.filter(['type','Student ID','budget','Per Student Budget',\
                                             'math_score','reading_score','% Passing Math',\
                                            '% Passing Reading','% Overall Passing Rate'])
# sorting decending
Top_Performing_Schools = school_summary.sort_values(by='% Overall Passing Rate',ascending = False)

# formatting
Top_Performing_Schools['Per Student Budget']=Top_Performing_Schools['Per Student Budget'].map('${:,.2f}'.format)
Top_Performing_Schools['budget']=Top_Performing_Schools['budget'].map('${:,.2f}'.format)
Top_Performing_Schools = Top_Performing_Schools.rename (columns={'budget':'Total Budget'})

#print
Top_Performing_Schools.head()

,type,Student ID,Total Budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [156]:
# Bottom Performing Schools (By Passing Rate)

# sorting decending
Bottom_Performing_Schools = school_summary.sort_values(by='% Overall Passing Rate',ascending = True)

# formatting
Bottom_Performing_Schools['Per Student Budget']= Bottom_Performing_Schools['Per Student Budget'].map('${:,.2f}'.format)
Bottom_Performing_Schools['budget']=Bottom_Performing_Schools['budget'].map('${:,.2f}'.format)
Bottom_Performing_Schools = Bottom_Performing_Schools.rename (columns={'budget':'Total Budget'})

#print
Bottom_Performing_Schools.head()

,type,Student ID,Total Budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [157]:
#  Math Scores by Grade
stu_ninth = student_data.loc[student_data['grade']=='9th']
stu_ninth = stu_ninth.groupby(['school_name']).agg({'math_score':'mean'})

stu_tenth = student_data.loc[student_data['grade']=='10th']
stu_tenth = stu_tenth.groupby(['school_name']).agg({'math_score':'mean'})

stu_11th = student_data.loc[student_data['grade']=='11th']
stu_11th = stu_11th.groupby(['school_name']).agg({'math_score':'mean'})

stu_12th = student_data.loc[student_data['grade']=='12th']
stu_12th = stu_12th.groupby(['school_name']).agg({'math_score':'mean'})

#merge
Math_by_grade = stu_ninth.merge(stu_tenth, on = 'school_name',suffixes=('_9th','_10th'))
Math_by_grade = Math_by_grade.merge(stu_11th, on ='school_name')
Math_by_grade = Math_by_grade.rename(columns = {'math_score':'math_score_11th'})
Math_by_grade = Math_by_grade.merge(stu_12th, on ='school_name')
Math_by_grade = Math_by_grade.rename(columns = {'math_score':'math_score_12th'})
Math_by_grade = Math_by_grade.rename(columns ={'math_score_9th':'9th','math_score_10th':'10th',\
                                             'math_score_11th':'11th','math_score_12th':'12th'} )
Math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [158]:
# Reading Score by Grade

stu_ninth2 = student_data.loc[student_data['grade']=='9th']
stu_ninth2 = stu_ninth.groupby(['school_name']).agg({'reading_score':'mean'})

stu_tenth2 = student_data.loc[student_data['grade']=='10th']
stu_tenth2 = stu_tenth.groupby(['school_name']).agg({'reading_score':'mean'})

stu_11th2 = student_data.loc[student_data['grade']=='11th']
stu_11th2 = stu_11th.groupby(['school_name']).agg({'reading_score':'mean'})

stu_12th2 = student_data.loc[student_data['grade']=='12th']
stu_12th2 = stu_12th.groupby(['school_name']).agg({'reading_score':'mean'})

#merge
Reading_by_grade = stu_ninth2.merge(stu_tenth2, on = 'school_name',suffixes=('_9th','_10th'))
Reading_by_grade = Reading_by_grade.merge(stu_11th2, on ='school_name')
Reading_by_grade = Reading_by_grade.rename(columns = {'reading_score':'reading_score_11th'})
Reading_by_grade = Reading_by_grade.merge(stu_12th2, on ='school_name')
Reading_by_grade = Reading_by_grade.rename(columns = {'reading_score':'reading_score_12th'})
Reading_by_grade = Reading_by_grade.rename(columns ={'reading_score_9th':'9th','reading_score_10th':'10th',\
                                            'reading_score_11th':'11th','reading_score_12th':'12th'} )
Reading_by_grade


/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


KeyError: 'reading_score'

In [ ]:
# Scores by School Spending
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#Use 4 reasonable bins to group school spending.

#bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#add "spending range" column
stu_by_school_merge['Spending Ranges (Per Student)'] = pd.cut(stu_by_school_merge['Per Student Budget'], bins = spending_bins, labels = group_names)

#group by 'spending range'
score_by_school_Spending = stu_by_school_merge.groupby('Spending Ranges (Per Student)')\
.agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean','% Passing Reading':'mean','% Overall Passing Rate':'mean'})

#print
score_by_school_Spending

In [ ]:
# Scores by School Size

#bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# add "school size" column
stu_by_school_merge['School Size'] = pd.cut(stu_by_school_merge['size'], bins = size_bins,labels = group_names)

#group by "school size"
stu_by_school_size = stu_by_school_merge.groupby('School Size')\
.agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean','% Passing Reading':'mean','% Overall Passing Rate':'mean'})

#print
stu_by_school_size

In [ ]:
# Scores by School Type

stu_by_school_type = stu_by_school_merge.groupby('type')\
.agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean','% Passing Reading':'mean','% Overall Passing Rate':'mean'})

stu_by_school_type